# Idea

- Using model YOLOv8 with nano version to classify images.
- Label 91 images which are divided into 3 classes:
  - Stamp-overlap: Stamp is overlap money
  - Money-overlap: Money is overlap stamp
  - Nothing-overlap: Stamp may be overlap in something but it is absolutely money.

#Setup thư viện

In [76]:
from IPython.display import clear_output

In [77]:
!pip install ultralytics
!pip install cvzone
!pip install roboflow
clear_output()

In [78]:
'''Connect to drive for loading dataset and save model'''
# from google.colab import drive
# drive.mount('/content/drive')

# Training model


In [33]:
# Download dataset from https://universe.roboflow.com/isods-xhfnh/logo-digit-overlap/dataset/1

/content/drive/.shortcut-targets-by-id/19mbDgMZtpIY7oXUbKNVekQeHcy2t4TIF/YOLOv8_Finetune/Logo
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.219, to fix: `pip install ultralytics==8.0.196`

Version export complete for yolov8 format



Extracting Dataset Version Zip to Logo-digit-overlap-2 in yolov8:: 100%|██████████| 439/439 [00:02<00:00, 197.71it/s]


In [41]:
# Create training yaml file
# %cd /content
!rm mydataset.yaml # If there is set-up file for training, it would be deleted

!echo 'train: /content/drive/MyDrive/YOLOv8_Finetune/Logo/Logo-digit-overlap-1/train' >> mydataset.yaml # Repace path of data for training
!echo 'val: /content/drive/MyDrive/YOLOv8_Finetune/Logo/Logo-digit-overlap-1/valid' >> mydataset.yaml # Repace path of data for valaidating

!echo 'nc: 3' >> mydataset.yaml # Number of class
!echo "names: ['money-overlap','nothing-overlap','stamp-overlap']" >> mydataset.yaml

/content


In [42]:
# Train với CLI
%cd /content/drive/MyDrive/YOLOv8_Finetune/Logo # cd to folder you want to save data
!yolo task=detect mode=train model=yolov8n.pt data=mydataset.yaml epochs=30 imgsz=640 # Train model

/content/drive/.shortcut-targets-by-id/19mbDgMZtpIY7oXUbKNVekQeHcy2t4TIF/YOLOv8_Finetune/Logo
Ultralytics YOLOv8.0.219 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv8_Finetune/Logo/runs/detect/train9/weights/last.pt, data=/content/mydataset.yaml, epochs=30, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, s

# Predict

In [81]:
# Train với Python API
from ultralytics import YOLO
import cvzone,cv2, math, os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [44]:
def imshow(img, showAxis = False, size=(20,10)):
  plt.figure(figsize=size)
  if not showAxis: plt.axis('off')
  if len(img.shape) == 3: plt.imshow(img[:,:,::-1])
  else: plt.imshow(img, cmap='gray')

In [82]:
model = YOLO("./model/best.pt") # Load model

In [95]:
def detect_model(file, model):
  img = cv2.imread(file)
  results = model.predict(file)
  className = model.names

  for r in results:
    boxes = r.boxes
    object_masks = r.masks

    for box_index, box in enumerate(boxes):
      cls = int(box.cls[0])
      x1, y1, x2, y2 = list(map(int, box.xyxy[0]))

      w,h = x2-x1, y2-y1
      conf = math.ceil((box.conf[0]*100))/100
      if conf > 0.8:
        cvzone.cornerRect(img, (x1,y1,w,h), l=9)
        cvzone.putTextRect(img,f"{className[cls]}|{conf}",(max(0,x1-40),max(20,y1-20)),scale=2,thickness=2)
  return img

In [96]:
def stamp_detect(dst:str, src_dirs:list, model):
  for src in tqdm(src_dirs,desc="src_dirs"):
    type = os.path.split(src)
    save_dir = os.path.join(dst,type[-1])
    os.makedirs(save_dir,exist_ok=True)
    for root, dir, files in os.walk(src):
      for file in tqdm(files,desc="files",leave=True):
        try:
          imgFile = os.path.join(root,file)
          img = detect_model(imgFile,model)
          cv2.imwrite(os.path.join(save_dir,file), img)
        except Exception as e:
          print(os.path.join(root,file))
          raise e

In [97]:
stamp_detect("/content/drive/MyDrive/YOLOv8_Finetune/Logo/results",["/content/drive/MyDrive/YOLOv8_Finetune/Logo/C008"],model)

src_dirs:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/161 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/KJSZJ202304030043412970832.jpg: 640x320 1 stamp-overlap, 11.7ms
Speed: 2.7ms preprocess, 11.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/KJSZJ202304022020219171718.jpg: 640x416 1 stamp-overlap, 7.8ms
Speed: 1.9ms preprocess, 7.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/KJSZJ202304061711508713097.jpg: 640x320 1 stamp-overlap, 7.7ms
Speed: 1.9ms preprocess, 7.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/KJSZJ202304032351068950769.jpg: 640x320 1 stamp-overlap, 7.2ms
Speed: 1.9ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/KJSZJ202304050621174647802.jpg: 640x320 1 stamp-overlap, 9.6ms

files:   0%|          | 0/4 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/advance/cl4sjuk3n5h8mgbdclq0_615556.jpg: 640x288 1 stamp-overlap, 9.9ms
Speed: 2.6ms preprocess, 9.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 288)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/advance/cl4ruqc3n5h9o93nlc70_962517.jpg: 640x288 1 stamp-overlap, 11.9ms
Speed: 2.5ms preprocess, 11.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 288)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/advance/cl4qv5k3n5hf2gppkif0_313251.jpg: 640x320 1 stamp-overlap, 10.5ms
Speed: 3.0ms preprocess, 10.5ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 /content/drive/MyDrive/YOLOv8_Finetune/Logo/C008/advance/cl4snes3n5hfnbco9uvg_987320.jpg: 640x288 1 stamp-overlap, 10.2ms
Speed: 2.7ms preprocess, 10.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 288)
